In [50]:
import pandas as pd
import re
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
import pickle

In [49]:
original_df = pd.read_csv('Chicago_Airbnb_Listing_Dataset.csv')
airbnb_df = original_df
airbnb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6544 entries, 0 to 6543
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      6544 non-null   int64  
 1   neighbourhood_cleansed  6544 non-null   object 
 2   latitude                6544 non-null   float64
 3   longitude               6544 non-null   float64
 4   room_type               6544 non-null   object 
 5   accommodates            6544 non-null   int64  
 6   bathrooms_text          6530 non-null   object 
 7   bedrooms                5965 non-null   float64
 8   beds                    6345 non-null   float64
 9   price                   6544 non-null   object 
 10  minimum_nights          6544 non-null   int64  
 11  maximum_nights          6544 non-null   int64  
 12  minimum_minimum_nights  6544 non-null   int64  
 13  maximum_minimum_nights  6544 non-null   int64  
 14  minimum_maximum_nights  6544 non-null   

In [3]:
airbnb_df['price'] = airbnb_df['price'].str.replace('$', '')
airbnb_df['price'] = airbnb_df['price'].str.replace(',', '').astype(float)

C:\Users\Rus\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [4]:
airbnb_df.drop(airbnb_df[airbnb_df.price > 1000].index, inplace=True)

In [5]:
airbnb_df.head()

,id,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm
0,2384,Hyde Park,41.78790,-87.58780,Private room,1,1 shared bath,1.0,1.0,95.0,3,89,3,3,1125,1125,3.0,1125.0
1,7126,West Town,41.90166,-87.68021,Entire home/apt,2,1 bath,1.0,1.0,65.0,2,60,2,2,1125,1125,2.0,1125.0
2,10945,Lincoln Park,41.91196,-87.63981,Entire home/apt,4,1 bath,2.0,2.0,127.0,4,180,4,4,180,180,4.0,180.0
3,12140,Lincoln Park,41.92357,-87.64947,Private room,2,1 private bath,1.0,2.0,329.0,2,7,2,2,7,7,2.0,7.0
4,22362,West Town,41.89730,-87.65889,Entire home/apt,4,2 baths,2.0,2.0,105.0,121,1125,121,121,1125,1125,121.0,1125.0


In [6]:
neighbourhoods = airbnb_df['neighbourhood_cleansed'].value_counts()
neighbourhoods

Near North Side    749
West Town          676
Lake View          553
Near West Side     402
Logan Square       391
                  ... 
Edison Park          1
Avalon Park          1
Burnside             1
Gage Park            1
West Elsdon          1
Name: neighbourhood_cleansed, Length: 76, dtype: int64

In [7]:
neighbourhood_list = neighbourhoods.index
neighbourhood_list
neighbourhood_num = np.arange(len(neighbourhood_list))

In [20]:
room_type_list = airbnb_df['room_type'].value_counts().index
accommodates_list = airbnb_df['accommodates'].value_counts().index
bathroom_list = airbnb_df['bathrooms_text'].value_counts().index
bedroom_list = airbnb_df['bedrooms'].value_counts().index
beds_list = airbnb_df['beds'].value_counts().index

In [14]:
neighbourhood_list

Index(['Near North Side', 'West Town', 'Lake View', 'Near West Side',
       'Logan Square', 'Loop', 'Lincoln Park', 'Uptown', 'Near South Side',
       'Lower West Side', 'Irving Park', 'Edgewater', 'Bridgeport', 'Avondale',
       'Rogers Park', 'North Center', 'East Garfield Park', 'Grand Boulevard',
       'Hyde Park', 'Lincoln Square', 'South Shore', 'Humboldt Park',
       'West Ridge', 'Woodlawn', 'Portage Park', 'Douglas', 'Albany Park',
       'Kenwood', 'Armour Square', 'Austin', 'Greater Grand Crossing',
       'South Lawndale', 'Jefferson Park', 'Washington Park', 'North Lawndale',
       'Mckinley Park', 'Norwood Park', 'Dunning', 'Calumet Heights',
       'South Chicago', 'West Garfield Park', 'Belmont Cragin',
       'Brighton Park', 'North Park', 'Hermosa', 'Englewood', 'New City',
       'Oakland', 'Garfield Ridge', 'Chatham', 'Auburn Gresham', 'Ohare',
       'Ashburn', 'Pullman', 'East Side', 'Forest Glen', 'Washington Heights',
       'Roseland', 'West Englewood', '

In [8]:
# neighborhood_dict = dict(zip(neighbourhood_list,neighbourhood_num))
neighborhood_dict = dict(zip(neighbourhoods.index,np.arange(len(neighbourhoods.index))))
neighborhood_dict

{'Near North Side': 0,
 'West Town': 1,
 'Lake View': 2,
 'Near West Side': 3,
 'Logan Square': 4,
 'Loop': 5,
 'Lincoln Park': 6,
 'Uptown': 7,
 'Near South Side': 8,
 'Lower West Side': 9,
 'Irving Park': 10,
 'Edgewater': 11,
 'Bridgeport': 12,
 'Avondale': 13,
 'Rogers Park': 14,
 'North Center': 15,
 'East Garfield Park': 16,
 'Grand Boulevard': 17,
 'Hyde Park': 18,
 'Lincoln Square': 19,
 'South Shore': 20,
 'Humboldt Park': 21,
 'West Ridge': 22,
 'Woodlawn': 23,
 'Portage Park': 24,
 'Douglas': 25,
 'Albany Park': 26,
 'Kenwood': 27,
 'Armour Square': 28,
 'Austin': 29,
 'Greater Grand Crossing': 30,
 'South Lawndale': 31,
 'Jefferson Park': 32,
 'Washington Park': 33,
 'North Lawndale': 34,
 'Mckinley Park': 35,
 'Norwood Park': 36,
 'Dunning': 37,
 'Calumet Heights': 38,
 'South Chicago': 39,
 'West Garfield Park': 40,
 'Belmont Cragin': 41,
 'Brighton Park': 42,
 'North Park': 43,
 'Hermosa': 44,
 'Englewood': 45,
 'New City': 46,
 'Oakland': 47,
 'Garfield Ridge': 48,
 'Ch

In [9]:
cleaning_df = airbnb_df
cleaning_df['neighbourhood_cleansed'] = cleaning_df['neighbourhood_cleansed'].apply(lambda x:neighborhood_dict[x])
cleaning_df.head()

,id,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm
0,2384,18,41.78790,-87.58780,Private room,1,1 shared bath,1.0,1.0,95.0,3,89,3,3,1125,1125,3.0,1125.0
1,7126,1,41.90166,-87.68021,Entire home/apt,2,1 bath,1.0,1.0,65.0,2,60,2,2,1125,1125,2.0,1125.0
2,10945,6,41.91196,-87.63981,Entire home/apt,4,1 bath,2.0,2.0,127.0,4,180,4,4,180,180,4.0,180.0
3,12140,6,41.92357,-87.64947,Private room,2,1 private bath,1.0,2.0,329.0,2,7,2,2,7,7,2.0,7.0
4,22362,1,41.89730,-87.65889,Entire home/apt,4,2 baths,2.0,2.0,105.0,121,1125,121,121,1125,1125,121.0,1125.0


In [10]:
# airbnb_df['bathrooms_text'] = airbnb_df['bathrooms_text'].str.replace(' shared bath', '')
cleaning_df['bathrooms_text'].value_counts()

1 bath               3296
2 baths               968
1 shared bath         738
1 private bath        451
2 shared baths        202
1.5 baths             187
2.5 baths             157
1.5 shared baths      108
3 baths               108
3.5 baths              66
4 baths                36
3 shared baths         33
2.5 shared baths       32
4 shared baths         19
4.5 baths              15
0 shared baths          9
0 baths                 5
6 baths                 5
11 shared baths         4
5 baths                 4
Private half-bath       3
Shared half-bath        3
7 baths                 2
Half-bath               2
11.5 shared baths       1
6.5 baths               1
8 shared baths          1
12.5 baths              1
Name: bathrooms_text, dtype: int64

In [11]:
cleaning_df['bathrooms_text'] = cleaning_df['bathrooms_text'].str.replace('-bath', '0.5')
cleaning_df['bathrooms_text'] = cleaning_df['bathrooms_text'].str.extract('(\d*\.\d+|\d+)', expand=False)
cleaning_df['bathrooms_text'] = cleaning_df['bathrooms_text'].astype(float)
cleaning_df = cleaning_df.dropna()

In [13]:
# working_df = pd.get_dummies(cleaning_df)
working_df = cleaning_df.drop(['price','id','latitude','longitude','minimum_minimum_nights','maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm','maximum_nights_avg_ntm', 'minimum_nights', 'maximum_nights'], axis=1)
working_df

,neighbourhood_cleansed,room_type,accommodates,bathrooms_text,bedrooms,beds
0,18,Private room,1,1.0,1.0,1.0
1,1,Entire home/apt,2,1.0,1.0,1.0
2,6,Entire home/apt,4,1.0,2.0,2.0
3,6,Private room,2,1.0,1.0,2.0
4,1,Entire home/apt,4,2.0,2.0,2.0
...,...,...,...,...,...,...
6539,1,Entire home/apt,2,1.0,1.0,1.0
6540,1,Entire home/apt,6,1.0,3.0,3.0
6541,2,Entire home/apt,6,2.0,2.0,2.0
6542,2,Entire home/apt,6,2.0,2.0,2.0


In [23]:
X = pd.get_dummies(working_df)
y = cleaning_df['price']
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5739 entries, 0 to 6543
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   neighbourhood_cleansed     5739 non-null   int32  
 1   accommodates               5739 non-null   int64  
 2   bathrooms_text             5739 non-null   float64
 3   bedrooms                   5739 non-null   float64
 4   beds                       5739 non-null   float64
 5   room_type_Entire home/apt  5739 non-null   uint8  
 6   room_type_Hotel room       5739 non-null   uint8  
 7   room_type_Private room     5739 non-null   uint8  
 8   room_type_Shared room      5739 non-null   uint8  
dtypes: float64(3), int32(1), int64(1), uint8(4)
memory usage: 269.0 KB


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
scaler = MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [25]:
model = RandomForestRegressor(random_state=42, n_estimators=100, max_depth=3)

In [26]:
# Fit the model on training data
model.fit(X_train_scaled, y_train)
        
# Predict results
y_test_preds = model.predict(X_test_scaled) 

# Evaluate the result
rsquared_score = r2_score(y_test, y_test_preds)
print('R sequared = ', rsquared_score)

R sequared =  0.4702645502163091


In [ ]:
# X_nn = working_df.drop(['price','id'], axis=1)


In [30]:
number_input_features = 9 #len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation=None))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                800       
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 3,261
Trainable params: 3,261
Non-trainable params: 0
_________________________________________________________________


In [31]:
nn.compile(loss="mean_absolute_error", optimizer="adam")

In [32]:
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,validation_split=0.1)

Epoch 1/100
122/122 [==============================] - 1s 2ms/step - loss: 144.1147 - val_loss: 127.4939
Epoch 2/100
122/122 [==============================] - 0s 1ms/step - loss: 87.4892 - val_loss: 69.8323
Epoch 3/100
122/122 [==============================] - 0s 1ms/step - loss: 64.3122 - val_loss: 65.1513
Epoch 4/100
122/122 [==============================] - 0s 1ms/step - loss: 61.5432 - val_loss: 63.3509
Epoch 5/100
122/122 [==============================] - 0s 1ms/step - loss: 60.3365 - val_loss: 61.9298
Epoch 6/100
122/122 [==============================] - 0s 1ms/step - loss: 59.3606 - val_loss: 60.6235
Epoch 7/100
122/122 [==============================] - 0s 1ms/step - loss: 58.5984 - val_loss: 59.5939
Epoch 8/100
122/122 [==============================] - 0s 1ms/step - loss: 57.9893 - val_loss: 58.9392
Epoch 9/100
122/122 [==============================] - 0s 1ms/step - loss: 57.6317 - val_loss: 58.4166
Epoch 10/100
122/122 [==============================] - 0s 1ms/step - l

Epoch 80/100
122/122 [==============================] - 0s 1ms/step - loss: 53.6213 - val_loss: 54.3588
Epoch 81/100
122/122 [==============================] - 0s 1ms/step - loss: 53.5516 - val_loss: 54.4373
Epoch 82/100
122/122 [==============================] - 0s 1ms/step - loss: 53.6153 - val_loss: 54.8187
Epoch 83/100
122/122 [==============================] - 0s 1ms/step - loss: 53.6063 - val_loss: 54.3724
Epoch 84/100
122/122 [==============================] - 0s 1ms/step - loss: 53.5799 - val_loss: 54.4136
Epoch 85/100
122/122 [==============================] - 0s 1ms/step - loss: 53.4601 - val_loss: 54.5014
Epoch 86/100
122/122 [==============================] - 0s 1ms/step - loss: 53.4139 - val_loss: 54.3244
Epoch 87/100
122/122 [==============================] - 0s 1ms/step - loss: 53.4823 - val_loss: 54.4161
Epoch 88/100
122/122 [==============================] - 0s 1ms/step - loss: 53.4227 - val_loss: 54.6257
Epoch 89/100
122/122 [==============================] - 0s 1ms/s

In [33]:
# price_bins = [0.0, 49.0, 99.0, 149.0, 199.0, 249.0, 299.0, 
#               349.0, 399.0, 449.0, 499.0, 549.0, 
#               599.0, 649.0, 699.0, 749.0, 799.0,
#              849.0, 899.0, 949.0, 999.0, 1000.0]

# price_labels = ['0-49','50-99','100-149', '150-199', '200-249', '250-299', '300-349', '350-399','400-449', '450-499',
#              '500-549', '550-599', '600-649', '650-699', '700-749', '750-799', '800-849', '850-899', '900-949', '950-999', 
#              '1000+']

# price_bins = [0.0, 99.0, 199.0, 299.0,
#               399.0, 499.0, 599.0, 699.0, 799.0, 
#               899.0, 999.0, 1000.0]

# price_labels = ['0-99','100-199', '200-299', '300-399','400-499',
#              '500-599', '600-699','700-799', '800-899', '900-999', 
#              '1000+']

price_bins = [0.0, 49.0, 74.0, 99.0, 124.0, 149.0, 199.0, 299.0, 100000.0]

price_labels = ['0-49','50-74','75-99','100-124','125-149','150-199', '200-299', '300+']

In [34]:
class_airbnb_df = pd.read_csv('Chicago_Airbnb_Listing_Dataset.csv')
class_airbnb_df['price'] = class_airbnb_df['price'].str.replace('$', '')
class_airbnb_df['price'] = class_airbnb_df['price'].str.replace(',', '').astype(float)
class_airbnb_df['bathrooms_text'] = class_airbnb_df['bathrooms_text'].str.replace('-bath', '0.5')
class_airbnb_df['bathrooms_text'] = class_airbnb_df['bathrooms_text'].str.extract('(\d*\.\d+|\d+)', expand=False)
class_airbnb_df['bathrooms_text'] = class_airbnb_df['bathrooms_text'].astype(float)
class_airbnb_df["price_range"] = pd.cut(class_airbnb_df["price"], price_bins, labels=price_labels)
# airbnb_df.info
counts = class_airbnb_df["price_range"].value_counts()
total = counts/counts.sum() 

C:\Users\Rus\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [35]:
RFC_df = class_airbnb_df
RFC_df = RFC_df.dropna()
class_df = RFC_df.drop(['price','price_range','id','minimum_minimum_nights',
                           'maximum_minimum_nights', 'minimum_maximum_nights','maximum_maximum_nights',
                           'minimum_nights_avg_ntm','maximum_nights_avg_ntm', 'minimum_nights','maximum_nights'], axis=1)
X_class = pd.get_dummies(class_df)
# X_class['room_type']  = LabelEncoder().fit_transform(X_class['room_type'])
y_class = LabelEncoder().fit_transform(RFC_df['price_range'])

In [36]:
airbnb_df.head()

,id,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm
0,2384,18,41.78790,-87.58780,Private room,1,1.0,1.0,1.0,95.0,3,89,3,3,1125,1125,3.0,1125.0
1,7126,1,41.90166,-87.68021,Entire home/apt,2,1.0,1.0,1.0,65.0,2,60,2,2,1125,1125,2.0,1125.0
2,10945,6,41.91196,-87.63981,Entire home/apt,4,1.0,2.0,2.0,127.0,4,180,4,4,180,180,4.0,180.0
3,12140,6,41.92357,-87.64947,Private room,2,1.0,1.0,2.0,329.0,2,7,2,2,7,7,2.0,7.0
4,22362,1,41.89730,-87.65889,Entire home/apt,4,2.0,2.0,2.0,105.0,121,1125,121,121,1125,1125,121.0,1125.0


In [37]:
X_class.head()

,latitude,longitude,accommodates,bathrooms_text,bedrooms,beds,neighbourhood_cleansed_Albany Park,neighbourhood_cleansed_Archer Heights,neighbourhood_cleansed_Armour Square,neighbourhood_cleansed_Ashburn,...,neighbourhood_cleansed_West Garfield Park,neighbourhood_cleansed_West Lawn,neighbourhood_cleansed_West Pullman,neighbourhood_cleansed_West Ridge,neighbourhood_cleansed_West Town,neighbourhood_cleansed_Woodlawn,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room
0,41.78790,-87.58780,1,1.0,1.0,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,41.90166,-87.68021,2,1.0,1.0,1.0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,41.91196,-87.63981,4,1.0,2.0,2.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,41.92357,-87.64947,2,1.0,1.0,2.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,41.89730,-87.65889,4,2.0,2.0,2.0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0


In [38]:
# Split Data to test and train
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, random_state=1)

#Scale data
scaler = MinMaxScaler().fit(X_train_class)
X_train_scaled = scaler.transform(X_train_class)
X_test_scaled = scaler.transform(X_test_class)

In [39]:
clf = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=3).fit(X_train_scaled, y_train_class)

In [40]:
print(f'Training Score: {clf.score(X_train_scaled, y_train_class)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test_class)}')

Training Score: 0.3499884872208151
Testing Score: 0.337707182320442


In [41]:
prediction = clf.predict([X_test_scaled[1]])
prediction

array([7])

In [42]:
print([X_test_scaled[1]])

[array([0.67185375, 0.49163557, 0.2       , 0.08      , 0.09090909,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.    

In [43]:
pred_val = prediction.tolist()

In [44]:
number_input_features_x = 86
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 30
# hidden_nodes_layer4 = 30

nn_class = tf.keras.models.Sequential()

# First hidden layer
nn_class.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features_x, activation="relu")
)

# Second hidden layer
nn_class.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

nn_class.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# nn_class.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn_class.add(tf.keras.layers.Dense(units=8, activation='sigmoid'))

# Check the structure of the model
nn_class.summary()

nn_class.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0001,
#     epsilon=1e-07,
    name='Adam'
), metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                6960      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 30)                930       
                                                                 
 dense_9 (Dense)             (None, 8)                 248       
                                                                 
Total params: 10,568
Trainable params: 10,568
Non-trainable params: 0
_________________________________________________________________


In [45]:
y_reshape = y_train_class.reshape(-1,1)
y_reshape_test = y_test_class.reshape(-1,1)
y_nn_enc = OneHotEncoder().fit(y_reshape)
y_train_nn = y_nn_enc.transform(y_reshape).toarray()
y_test_nn = y_nn_enc.transform(y_reshape_test).toarray()

In [46]:
fit_class = nn_class.fit(X_train_scaled,y_train_nn,epochs=100,validation_split=0.1)

Epoch 1/100
123/123 [==============================] - 1s 3ms/step - loss: 2.0621 - accuracy: 0.2016 - val_loss: 2.0388 - val_accuracy: 0.2345
Epoch 2/100
123/123 [==============================] - 0s 1ms/step - loss: 2.0233 - accuracy: 0.2334 - val_loss: 1.9906 - val_accuracy: 0.2759
Epoch 3/100
123/123 [==============================] - 0s 2ms/step - loss: 1.9706 - accuracy: 0.2449 - val_loss: 1.9267 - val_accuracy: 0.2644
Epoch 4/100
123/123 [==============================] - 0s 2ms/step - loss: 1.9129 - accuracy: 0.2607 - val_loss: 1.8688 - val_accuracy: 0.2713
Epoch 5/100
123/123 [==============================] - 0s 2ms/step - loss: 1.8658 - accuracy: 0.2625 - val_loss: 1.8286 - val_accuracy: 0.2759
Epoch 6/100
123/123 [==============================] - 0s 2ms/step - loss: 1.8287 - accuracy: 0.2779 - val_loss: 1.7977 - val_accuracy: 0.2966
Epoch 7/100
123/123 [==============================] - 0s 2ms/step - loss: 1.7995 - accuracy: 0.2966 - val_loss: 1.7720 - val_accuracy: 0.3333

Epoch 58/100
123/123 [==============================] - 0s 1ms/step - loss: 1.5304 - accuracy: 0.3710 - val_loss: 1.5184 - val_accuracy: 0.4000
Epoch 59/100
123/123 [==============================] - 0s 1ms/step - loss: 1.5289 - accuracy: 0.3680 - val_loss: 1.5179 - val_accuracy: 0.3954
Epoch 60/100
123/123 [==============================] - 0s 1ms/step - loss: 1.5286 - accuracy: 0.3738 - val_loss: 1.5206 - val_accuracy: 0.4000
Epoch 61/100
123/123 [==============================] - 0s 1ms/step - loss: 1.5282 - accuracy: 0.3744 - val_loss: 1.5148 - val_accuracy: 0.4023
Epoch 62/100
123/123 [==============================] - 0s 1ms/step - loss: 1.5270 - accuracy: 0.3723 - val_loss: 1.5192 - val_accuracy: 0.4023
Epoch 63/100
123/123 [==============================] - 0s 1ms/step - loss: 1.5267 - accuracy: 0.3726 - val_loss: 1.5175 - val_accuracy: 0.4023
Epoch 64/100
123/123 [==============================] - 0s 1ms/step - loss: 1.5257 - accuracy: 0.3690 - val_loss: 1.5188 - val_accuracy:

In [47]:
nn_class.evaluate(x=X_test_scaled,y=y_test_nn)

46/46 [==============================] - 0s 842us/step - loss: 1.5705 - accuracy: 0.3764


[1.5704739093780518, 0.3763812184333801]

In [51]:

pickle_out = open("RFC_Model","wb")
pickle.dump(clf, pickle_out)
pickle_out.close()

In [59]:
new_RF = pickle.load(open("RFC_Model","rb"))

In [61]:
new_RF.score(X_test_scaled, y_test_class)

0.337707182320442

In [55]:
nn_class.save("NN_TF_Model.h5")

In [56]:
new_model = tf.keras.models.load_model("NN_TF_Model.h5")

In [57]:
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                6960      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 30)                930       
                                                                 
 dense_9 (Dense)             (None, 8)                 248       
                                                                 
Total params: 10,568
Trainable params: 10,568
Non-trainable params: 0
_________________________________________________________________


In [58]:
new_model.evaluate(x=X_test_scaled,y=y_test_nn)

46/46 [==============================] - 0s 887us/step - loss: 1.5705 - accuracy: 0.3764


[1.5704739093780518, 0.3763812184333801]